# MY !FIRST LAB

### Script will take a stackoverflow issue and summarize it as a technical tutorial. 

Example links to use: 
    
https://stackoverflow.com/questions/14220321/how-do-i-return-the-response-from-an-asynchronous-call 
https://stackoverflow.com/questions/60174/how-can-i-prevent-sql-injection-in-php
https://stackoverflow.com/questions/1732348/regex-match-open-tags-except-xhtml-self-contained-tags

*Note: Issues must be answered preferebly by a lot of users.*


In [1]:
website = 'https://stackoverflow.com/questions/60174/how-can-i-prevent-sql-injection-in-php'

In [2]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

In [3]:
# Load environment variables in a file callwebsite_content .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook


In [4]:
openai = OpenAI()

# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.
# If it STILL doesn't work (horrors!) then please see the Troubleshooting notebook in this folder for full instructions

In [5]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermwebsite_contentiate Python" notebook

# Some websites newebsite_content you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [6]:
# Let's try one out. Change the website and add print statements to follow along.

website_content = Website(website)
print(website_content.title[:100])
print(website_content.text[:150])

mysql - How can I prevent SQL injection in PHP? - Stack Overflow
Skip to main content
Stack Overflow
About
Products
OverflowAI
Stack Overflow for Teams
Where developers & technologists share private knowledge with c


## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [2]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are a technical writer that analyzes the contents of a stackoverflow website issue containing a question and answer \
and provides a summary in the form of a technical tutorial , ignoring text that might be navigation related. \
Respond in markdown."

In [9]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += f""" 

    You are looking at a website titled {website_content.title}

    Create a technical tutorial baswebsite_content on the following Stack Overflow content:
    
    {website_content.text}


    The tutorial should include an introduction, problem statement, solution steps, and conclusion.
    Tutrial should be in markdown format.
    """
    user_prompt += website.text
    return user_prompt

In [10]:
print(user_prompt_for(website_content)[:500])

You are looking at a website titled mysql - How can I prevent SQL injection in PHP? - Stack Overflow 

    You are looking at a website titled mysql - How can I prevent SQL injection in PHP? - Stack Overflow

    Create a technical tutorial baswebsite_content on the following Stack Overflow content:

    Skip to main content
Stack Overflow
About
Products
OverflowAI
Stack Overflow for Teams
Where developers & technologists share private knowledge with coworkers
Advertising & Talent
Reach devs & t


## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]

To give you a preview, the next 2 cells make a rather simple call - we won't stretch the mighty GPT (yet!)

## And now let's build useful messages for GPT-4o-mini, using a function

In [11]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

## Time to bring it together - the API for OpenAI is very simple!

In [12]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [13]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [14]:
display_summary(website)

```markdown
# How to Prevent SQL Injection in PHP

## Introduction
SQL injection is a serious security vulnerability that can allow an attacker to interfere with the queries that your application makes to the database. By exploiting this vulnerability, an attacker can gain unauthorized access to sensitive data, manipulate data, and even execute administrative operations on the database. This tutorial will guide you on how to prevent SQL injection in your PHP applications through various best practices.

## Problem Statement
Consider the following PHP code that is vulnerable to SQL injection:

```php
$unsafe_variable = $_POST['user_input']; 
mysql_query("INSERT INTO `table` (`column`) VALUES ('$unsafe_variable')");
```

If a user were to input something like `value'); DROP TABLE table;--`, the query would become:

```sql
INSERT INTO `table` (`column`) VALUES('value'); DROP TABLE table;--');
```

This inserts an unwanted SQL command leading to disastrous effects on the database.

## Solution Steps

### 1. Use Prepared Statements
The best method to prevent SQL injection is to use prepared statements with parameterized queries. This separates SQL logic from data, ensuring that user input is treated as data, not executable code.

#### Using PDO
Here's how to use PDO in PHP:

```php
$dsn = 'mysql:dbname=dbtest;host=127.0.0.1;charset=utf8mb4';
$dbConnection = new PDO($dsn, 'user', 'password');
$dbConnection->setAttribute(PDO::ATTR_ERRMODE, PDO::ERRMODE_EXCEPTION);
$dbConnection->setAttribute(PDO::ATTR_EMULATE_PREPARES, false);

$stmt = $dbConnection->prepare('SELECT * FROM users WHERE name = :name');
$stmt->execute(['name' => $name]);

foreach ($stmt as $row) {
    // Process row
}
```

#### Using MySQLi
If you're using MySQLi, the syntax is slightly different:

```php
$dbConnection = new mysqli('127.0.0.1', 'username', 'password', 'test');
$dbConnection->set_charset('utf8mb4');

$stmt = $dbConnection->prepare('SELECT * FROM users WHERE name = ?');
$stmt->bind_param('s', $name); // 's' stands for string
$stmt->execute();
$result = $stmt->get_result();

while ($row = $result->fetch_assoc()) {
    // Process row
}
```

### 2. Properly Configure the Database Connection
When using PDO, ensure that emulated prepared statements are disabled. This is essential for real prepared statements to take effect.

Example configuration:
```php
$dbConnection->setAttribute(PDO::ATTR_EMULATE_PREPARES, false);
```

### 3. Validate Input Data
In addition to using prepared statements, you should validate and sanitize user inputs. Implementing whitelist validation can help by ensuring only expected values are processed.

For example, if you expect a sorting direction:
```php
$dir = !empty($_GET['dir']) && $_GET['dir'] === 'DESC' ? 'DESC' : 'ASC';
```

### 4. Limit Database Permissions
Restrict database user permissions to the minimum required for their role. For example, a user who only needs to read data should not have permissions to delete or alter it.

```sql
GRANT SELECT ON database TO 'username'@'localhost';
```

### 5. Regularly Update Your Codebase
Keep libraries and the PHP version you are using up-to-date. Deprecated functions and libraries often contain vulnerabilities that can be exploited.

## Conclusion
Preventing SQL injection in PHP applications requires a proactive approach. Using prepared statements ensures user input is handled securely, while validating data and limiting permissions fortifies your application against potential attacks. By implementing these best practices, you can significantly reduce the risk of SQL injection vulnerabilities in your applications.

For more in-depth information on SQL injection prevention techniques, consult the [OWASP SQL Injection Prevention Cheat Sheet](https://owasp.org/www-community/attacks/SQL_Injection).
```